# Работа с табличными данными в Pandas

In [ ]:
# Для оформления вывод убран при помощи ;

In [ ]:
import numpy as np
import pandas as pd

## Задание 1

In [ ]:
# Читаем данные
df = pd.read_csv(
    'data/info.csv',
    # Лучше сразу указать значения,
    # которые будем считать пропущенными
    na_values=['#Н/Д', '=#N/A']
)

# Убираем "\r\n" в названиях столбцов
df.columns = df.columns.str.replace('\r\n', ' ', regex=False)

# Переводим даты в формат
date_columns = [
    'Срок действия сертификата',
    'Информация об окончании срока технической поддержки, полученная от заявителя'
]
for i in date_columns:
    # Значение "бессрочно" заменяем для парсинга
    df[i].replace({'бессрочно': '01.01.2200'}, inplace=True)
    df[i] = pd.to_datetime(df[i], dayfirst=True)

In [ ]:
df;

## Задание 2

In [ ]:
# Заберем в отдельную переменную, чтобы не испортить данные
doc_names = df['Наименования документов, требованиям которых соответствует средство']

In [ ]:
# Уберем приставку
doc_names = doc_names.str.replace('Соответствует требованиям документов: ', '')
# Поделим строки по запятой
doc_names = doc_names.str.split(', ', expand=True)
doc_names = doc_names.values.flatten()

In [ ]:
# Уберем пропущенные значения
doc_names = doc_names[~pd.isna(doc_names)]
# Считаем уникальные значения
unique = np.unique(doc_names)

In [ ]:
len(unique)

In [ ]:
unique;

## Задание 3

In [ ]:
# Аналогичные шаги, но ...
doc_names = df['Наименования документов, требованиям которых соответствует средство']
doc_names = doc_names.str.replace('Соответствует требованиям документов: ', '')
# ... не делаем expand и заменим пропущенные
doc_names = doc_names.str.split(', ').fillna('-')

In [ ]:
# Можно попробовать .str.contains, но так надежнее
doc_1 = doc_names.map(lambda str_list: 'ТУ' in str_list)
doc_2 = doc_names.map(lambda str_list: 'Требования доверия(5)' in str_list)

filt = doc_1 & doc_1

In [ ]:
df[filt];

## Задание 4

In [ ]:
# normalize переводит к полночи
# Можно не использовать, тогда будет текущее время
today = pd.to_datetime('today').normalize()

In [ ]:
certificate_has_expired = df['Срок действия сертификата'] < today

In [ ]:
df[certificate_has_expired];

## Задание 5

In [ ]:
tech_support_active = df['Информация об окончании срока технической поддержки, полученная от заявителя'] > today
# Пропущенные значения вернут False
# Если интерпретировать их как "неизвестно", то их нужно отфильтровать

In [ ]:
df[certificate_has_expired & tech_support_active];

## Задание 6

In [ ]:
# Пропуски заменим
df['Испытательная лаборатория'] = df['Испытательная лаборатория'].fillna('Нет информации')

In [ ]:
# Вариант 1

# В регулярных выражениях | означает "или"
firm_types = ['ООО', 'ЗАО', 'ПАО', 'АО']
regex_or = '|'.join(firm_types)

filt_regex = df['Испытательная лаборатория'].str.contains(regex_or, regex=True)

In [ ]:
# Вариант 2

# Используем lambda + map
# В некоторых случаях такой вариант может быть проще

str_func = lambda string: any(firm_type in string for firm_type in firm_types)
filt_lambda = df['Испытательная лаборатория'].map(str_func)

In [ ]:
# Проверим, что результаты совпадают
(filt_regex == filt_lambda).all()

In [ ]:
df[filt_regex];